In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("exploration").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 13:43:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import col, cast,concat_ws,substring,lit,to_date

In [4]:
# loading the revised_final dataset from the data folder 

data_path = "./data/revised_final.csv"  
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [5]:
#loading goods_classifiation 
data_path = "./data/goods_classification.csv"  
df_goods = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/country_classification.csv"  
df_country = spark.read.csv(data_path, header=True, inferSchema=True)
data_path = "./data/services_classification.csv"  
df_services = spark.read.csv(data_path, header=True, inferSchema=True)

In [6]:
df.show()

+--------+-------+----+------------+------------+-------------+------+
|time_ref|account|code|country_code|product_type|        value|status|
+--------+-------+----+------------+------------+-------------+------+
|  202306|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|  202306|Exports|  00|          AG|       Goods|     351919.0|     F|
|  202306|Exports|  00|          AI|       Goods|      84762.0|     F|
|  202306|Exports|  00|          AL|       Goods|       3463.0|     F|
|  202306|Exports|  00|          AM|       Goods|     679586.0|     F|
|  202306|Exports|  00|          AO|       Goods|     464583.0|     F|
|  202306|Exports|  00|          AR|       Goods|    5943055.0|     F|
|  202306|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|  202306|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|  202306|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|  202306|Exports|  00|          AW|       Goods|     362396.0|     F|
|  202

In [7]:
df.printSchema()

root
 |-- time_ref: integer (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [8]:
#number of rows 
df.count()

700000

In [9]:
#we haveto change the time_ref into string so that we can use string methods for manipulation. 
df = df.withColumn("time_ref", col("time_ref").cast("string"))



In [10]:
#you can see that the time_ref has been changed into string 
df.printSchema()

root
 |-- time_ref: string (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [11]:
#concat_ws is used here to concat strings with delimiter "\"
#lit is used to append "30" as  a string at theend of the data.
df = df.withColumn("time_ref", concat_ws("/", substring(col("time_ref"), 1, 4), substring(col("time_ref"), 5, 2), lit('30')))


In [12]:
df.show()

+----------+-------+----+------------+------------+-------------+------+
|  time_ref|account|code|country_code|product_type|        value|status|
+----------+-------+----+------------+------------+-------------+------+
|2023/06/30|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|2023/06/30|Exports|  00|          AG|       Goods|     351919.0|     F|
|2023/06/30|Exports|  00|          AI|       Goods|      84762.0|     F|
|2023/06/30|Exports|  00|          AL|       Goods|       3463.0|     F|
|2023/06/30|Exports|  00|          AM|       Goods|     679586.0|     F|
|2023/06/30|Exports|  00|          AO|       Goods|     464583.0|     F|
|2023/06/30|Exports|  00|          AR|       Goods|    5943055.0|     F|
|2023/06/30|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|2023/06/30|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|2023/06/30|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|2023/06/30|Exports|  00|          AW|       Goods|

In [13]:
df.printSchema()

root
 |-- time_ref: string (nullable = false)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [14]:
#now to change the string type column to date format. we use to_date
df = df.withColumn("time_ref", to_date(col("time_ref"), "yyyy/MM/dd"))


In [15]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [16]:
null_count_accnt = df.filter(col("product_type").isNull()).count()
null_count_code = df.filter(col("product_type").isNull()).count()
null_count_country_code = df.filter(col("product_type").isNull()).count()
null_count_product_type = df.filter(col("product_type").isNull()).count()
null_count_value = df.filter(col("product_type").isNull()).count()
null_count_status = df.filter(col("status").isNull()).count()


In [17]:
print(null_count_accnt)
print(null_count_code)
print(null_count_country_code)
print(null_count_product_type)
print(null_count_status)
print(null_count_value)

0
0
0
0
0
0


<h3>Goods dataset</h3>


In [18]:
df_goods.printSchema()

root
 |-- NZHSC_Level_2_Code_HS4: integer (nullable = true)
 |-- NZHSC_Level_1_Code_HS2: string (nullable = true)
 |-- NZHSC_Level_2: string (nullable = true)
 |-- NZHSC_Level_1: string (nullable = true)
 |-- Status_HS4: string (nullable = true)



In [19]:
df_goods.show()

+----------------------+----------------------+--------------------+--------------------+----------+
|NZHSC_Level_2_Code_HS4|NZHSC_Level_1_Code_HS2|       NZHSC_Level_2|       NZHSC_Level_1|Status_HS4|
+----------------------+----------------------+--------------------+--------------------+----------+
|                   101|                    01|Horses, asses, mu...|       Animals; live|   Current|
|                   102|                    01|Bovine animals; live|       Animals; live|   Current|
|                   103|                    01|         Swine; live|       Animals; live|   Current|
|                   104|                    01|Sheep and goats; ...|       Animals; live|   Current|
|                   105|                    01|Poultry; live, fo...|       Animals; live|   Current|
|                   106|                    01|Animals; live, n....|       Animals; live|   Current|
|                   201|                    02|Meat of bovine an...|Meat and edible m...|  

In [20]:
df_services.printSchema()

root
 |-- code: string (nullable = true)
 |-- service_label: string (nullable = true)



In [21]:
df_services.show()

+-------+--------------------+
|   code|       service_label|
+-------+--------------------+
|    A12|            Services|
|  A1202|Maintenance and r...|
|  A1203|      Transportation|
| A12031|       Sea transport|
|A120312|             Freight|
|A120313|               Other|
| A12032|       Air transport|
|A120321|Air passenger tra...|
|A120322|Air freight trans...|
|A120323| Air other transport|
| A12034|Postal and courie...|
|  A1204|              Travel|
| A12041|     Business travel|
| A12042|     Personal travel|
|A120421|Education related...|
|A120422|Health related tr...|
|A120423|Other personal tr...|
|  A1205|Construction serv...|
|  A1206|Insurance and pen...|
| A12062|         Reinsurance|
+-------+--------------------+
only showing top 20 rows



In [22]:
df_country.printSchema()

root
 |-- country_code: string (nullable = true)
 |-- country_label: string (nullable = true)



In [23]:
df_country.show()

+------------+--------------------+
|country_code|       country_label|
+------------+--------------------+
|          AD|             Andorra|
|          AE|United Arab Emirates|
|          AF|         Afghanistan|
|          AG| Antigua and Barbuda|
|          AI|            Anguilla|
|          AL|             Albania|
|          AM|             Armenia|
|          AO|              Angola|
|          AQ|          Antarctica|
|          AR|           Argentina|
|          AS|     Samoa, American|
|          AT|             Austria|
|          AU|           Australia|
|          AW|               Aruba|
|          AZ|          Azerbaijan|
|          BA|Bosnia and Herzeg...|
|          BB|            Barbados|
|          BD|          Bangladesh|
|          BE|             Belgium|
|          BF|        Burkina Faso|
+------------+--------------------+
only showing top 20 rows

